# 5-2 DCGAN の損失関数・学習・生成の実装

## DCGAN の損失関数
DCGAN の損失関数は通常のクラス分類と同様に定義できる．（正確には Jensen-Shannon ダイバージェンスの話になるがここでは省略）  
入力される画像データが $x$ のとき，D の出力は $y = D(x)$ だが，出力 は前節で実装した D の出力にシグモイド関数がかかって，値が0から1に変換されているものとする．
正しいラベルは G が生成した偽データをラベル0，教師データをラベル1とすると D の出力が正答かどうかは $y^l(1-y)^{1-l}$ で表される．
$y^l(1-y)^{1-l}$ は正解ラベル $l$ と予測出力 $y$ の値が同じなら1，異なる間違った予測の場合は0になる．
実際には $y$ は0から1の間の値をとり極端に0や1にならないため，$y^l(1-y)^{1-l}$ も0から1の値になる．

この判定がミニバッチのデータ数 M 個分あるため，その同時確率は次で与えられる．
$$
    \prod_{i = 1}^{M} y_i^{l_i}(1-y_i)^{1-l_i}
$$
これの対数を取ると次のようになる．
$$
    \sum_{i = 1}^{M} [l_i log y_i + (1 - l_i) log (1 - y_i)]
$$
D はこの式（対数尤度）が最大となるように学習する．
最大化は実装が難しいので $-1$ を掛けて最小化問題に帰着する．  
この損失関数は torch.nn.BCEWithLogitsLoss() を使用して簡単に実装できる．（BCE は Binary Cross Entropy の略で2値分類の誤差関数）

In [2]:
import torch
import torch.nn as nn

# Generator
class Generator(nn.Module):
    def __init__(self, z_dim=20, image_size=64):
        super(Generator, self).__init__()
        
        self.layer1 = nn.Sequential(nn.ConvTranspose2d(z_dim, image_size * 8, kernel_size=4, stride=1), 
                                    nn.BatchNorm2d(image_size * 8), 
                                    nn.ReLU(inplace=True))
        self.layer2 = nn.Sequential(nn.ConvTranspose2d(image_size * 8, image_size * 4, kernel_size=4, stride=2, padding=1), 
                                    nn.BatchNorm2d(image_size * 4), 
                                    nn.ReLU(inplace=True))
        self.layer3 = nn.Sequential(nn.ConvTranspose2d(image_size * 4, image_size * 2, kernel_size=4, stride=2, padding=1), 
                                    nn.BatchNorm2d(image_size * 2), 
                                    nn.ReLU(inplace=True))
        self.layer4 = nn.Sequential(nn.ConvTranspose2d(image_size * 2, image_size, kernel_size=4, stride=2, padding=1), 
                                    nn.BatchNorm2d(image_size), 
                                    nn.ReLU(inplace=True))
        self.last = nn.Sequential(nn.ConvTranspose2d(image_size, 1, kernel_size=4, stride=2, padding=1), 
                                  nn.Tanh())
        
    
    def forward(self, z):
        out = self.layer1(z)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.last(out)
        
        return out
    

# Discriminator
class Discriminator(nn.Module):
    def __init__(self, z_dim=20, image_size=64):
        super(Discriminator, self).__init__()
        
        self.layer1 = nn.Sequential(nn.Conv2d(1, image_size, kernel_size=4, stride=2, padding=1), 
                                    nn.LeakyReLU(0.1, inplace=True))
        self.layer2 = nn.Sequential(nn.Conv2d(image_size, image_size * 2, kernel_size=4, stride=2, padding=1), 
                                    nn.LeakyReLU(0.1, inplace=True))
        self.layer3 = nn.Sequential(nn.Conv2d(image_size * 2, image_size * 4, kernel_size=4, stride=2, padding=1), 
                                    nn.LeakyReLU(0.1, inplace=True))
        self.layer4 = nn.Sequential(nn.Conv2d(image_size * 4, image_size * 8, kernel_size=4, stride=2, padding=1), 
                                    nn.LeakyReLU(0.1, inplace=True))
        self.last = nn.Conv2d(image_size * 8, 1, kernel_size=4, stride=1)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.last(out)
        
        return out

    
G = Generator()
D = Discriminator()

In [3]:
# D の誤差関数のイメージ実装
# maximize log(D(x)) + log(1 - D(G(z)))
# この段階では x が未定義なため動作はエラーになる

import torch
import torch.nn as nn

# 正解ラベルを作成
mini_batch_size = 2
label_real = torch.full((mini_batch_size,), 1)

# 偽ラベルを作成
label_fake = torch.full((mini_batch_size,), 0)

# 誤差関数を定義
criterion = nn.BCEWithLogitsLoss(reduction="mean")

# 真の画像を判定
d_out_real = D(x)

# 偽の画像を生成して判定
input_z = torch.randn(mini_batch_size, 20)
input_z = input_z.view(input_z.size(0), input_z.size(1), 1, 1)
fake_images = G(input_z)
d_out_fake = D(fake_images)

# 誤差を計算
d_loss_real = criterion(d_out_real.view(-1), label_real)
d_loss_fake = criterion(d_out_fake.view(-1), label_fake)
d_loss = d_loss_real + d_loss_fake

NameError: name 'x' is not defined

G は D を騙したいので G で生成した画像に対して D の判定が失敗方向になれば良い．
つまり D は G から生成された画像を正確に判定するために次式を最小化しようとしている．
$$
    -\sum_{i = 1}^{M} [l_i log y_i + (1 - l_i) log (1 - y_i)]
$$
よって，G の損失関数は次のようになる．
$$
    \sum_{i = 1}^{M} [l_i log y_i + (1 - l_i) log (1 - y_i)]
$$
ここで $l_i$ は偽画像は0なので第1項が消え，さらに $ 1 - l_i = 1, y = D(x)$ で，さらに G は $z_i$ から画像を生成するので，G のミニバッチでの損失関数は次で与えられる．
$$
    \sum_{i = 1}^{M} log\left(1-D(G(z_i))\right)
$$

しかし，上式で与えられる損失関数では学習がうまくいかない．
これは，初期段階の G が生成する画像は教師データと大きくかけ離れているため，未熟な D でもある程度正確な判定ができてしまうため，損失がほとんど0になってしまい学習が進めなくなるためである．  
そこで，$D(G(z_i))$が1と判定されれば良いと考え，DCGAN では G の損失関数を次のように定義する．
$$
    -\sum_{i = 1}^{M} log D(G(z_i))
$$
こうすることで，G がうまく D を騙せれば log の中身は1となり損失関数の値は0となる．
一方で D に見抜かれると log の中身は0から1の値となり損失関数は正の大きな値をとり，学習が進むようになる．

In [4]:
# G の誤差関数のイメージ実装
# maximize log(D(G(z)))
# x が未定義なので動作はエラーになります

# 偽の画像を生成して判定
input_z = torch.randn(mini_batch_size, 20)
input_z = input_z.view(input_z.size(0), input_z.size(1), 1, 1)
fake_images = G(input_z)
d_out_fake = D(fake_images)

# 誤差を計算
g_loss = criterion(d_out_fake.view(-1), label_real)

ここで，G の損失の計算に D での判定が入っている点に注目する．
損失をバックプロパゲーションしたとき，まず D を通ってから G に到達する．
もし，D の活性化関数に ReLU 関数を使うと，ReLU への入力が負であった場合に出力が0となってしまい，バックプロパゲーションもそこで止まってしまう．
これでは G の学習が進まないため D の活性化関数に LeakyReLU を使って G まで損失がでんぱするようにしている．